In [59]:
import numpy as np
import pyaudio
import time
from pyldpc import make_ldpc, encode, decode, get_message

# Ses kayıt parametreleri
CHUNK = 1024*1
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

# Taşıyıcı frekansı
tasiyici_frekans = 10000




n = 32
d_v = 8
d_c = 16
snr = 20
H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
k = G.shape[1]
#>>> v = np.random.randint(2, size=k)

# PyAudio nesnesi oluştur
p = pyaudio.PyAudio()

# Çıkış akışını aç
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    output=True,
    frames_per_buffer=CHUNK
)

# Gönderilecek veri (binary)
v = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0], dtype=np.int16)
y = encode(G, v, snr)
y=np.where(y<0,0,1)
genisletilmis_veri = np.repeat(v, CHUNK // len(v))
#print(genisletilmis_veri[:16])

# Taşıyıcı dalga oluşturma
t = np.arange(0, len(genisletilmis_veri)) / RATE
tasiyici_dalga = np.sin(2 * np.pi * tasiyici_frekans * t)
m=tasiyici_dalga * (genisletilmis_veri * 2 - 1) 
# Frekans modülasyonu
modüle_edilmis_dalga = m*32767 #np.where(m>0,m * 32767,m*-32767)

# Modüle edilmiş dalgayı ses cihazına gönderme
for i in range(100):
	stream.write(modüle_edilmis_dalga.astype(np.int16).tobytes())
	time.sleep(0.01)
# Akışı kapat
stream.stop_stream()

stream.close()
p.terminate()


ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred


print(len(sound_bits))


>>> import numpy as np
>>> from pyldpc import make_ldpc, encode, decode, get_message
>>> n = 32
>>> d_v = 4
>>> d_c = 8
>>> snr = 1000
>>> H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
>>> k = G.shape[1]
>>> v = np.random.randint(2, size=k)
>>> y = encode(G, v, snr)
>>> d = decode(H, y, snr)
>>> x = get_message(G, d)
>>> assert abs(x - v).sum() == 0

print(len(y))

print(len(v))

d = decode(H, y, 20)
x = get_message(G, d)
x


print(len(y))

y

aci = np.arcsin(0.5)  

In [19]:
modüle_edilmis_dalga

array([    -0.        , -21417.70013415, -32418.20922816, ...,
       -25324.60528974,  -5574.95445348,  16886.25575742])

In [20]:
m2=modüle_edilmis_dalga/32767

In [21]:
m2

array([-0.        , -0.65363628, -0.98935543, ..., -0.77286921,
       -0.1701393 ,  0.51534336])

In [22]:
t = np.arange(0, 2048) / RATE
tasiyici_dalga = np.sin(2 * np.pi * 5000 * t)
tasiyici_dalga=np.where(tasiyici_dalga==0,1e-10,tasiyici_dalga)




In [26]:
genis_veri=(m2/tasiyici_dalga +1)/2
genis_veri=np.where(genis_veri <=0,-1,genis_veri)


In [30]:
a=al(genis_veri)

a


array([1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])

In [32]:
def al(data):
	n = 32
	d_v = 4
	d_c = 8
	snr = 20
	H, G = make_ldpc(n, d_v, d_c, systematic=True, sparse=True)
	gruplar = [data[i:i+64] for i in range(0, len(data), 64)]
	ana_ort=np.abs(np.mean(data))
	ortalamalar =[]
	
	for grup in gruplar:
		ortalama=np.mean(grup)
		if ortalama <=0:
			ortalamalar.append(ortalama)
		else:
			ortalamalar.append(ortalama)
	ortalamalar=np.array(ortalamalar)	
	#print(ortalamalar[0])	
	d = decode(H, ortalamalar, snr)
	x=get_message(G, d)
			
	return x

In [558]:
genis_veri

array([ 0.5, -1. , -1. , ..., -1. , -1. , -1. ])

In [256]:
genisletilmis_veri

array([-1., -1., -1., ..., -1., -1., -1.])

In [2]:
modüle_edilmis_dalga

array([    -0.        , -21417.70013415, -32418.20922816, ...,
       -25324.60528974,  -5574.95445348,  16886.25575742])

In [9]:
np.sum(genis_veri >0)

1709

In [31]:
genis_veri

array([ 0.5, -1. , -1. , ...,  1. ,  1. ,  1. ])